San Francisco Fire Calls

In [1]:
## Cargamos los paquetes y libresrias
import findspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [10]:
## Creamos la Sparksesion y Definimos la ruta

spark = (SparkSession.builder.appName("Example 3.7").getOrCreate())

sf_fire_file = "C:/Users/javier.martin/Curso Spark/BD/sf-fire-calls.csv"

In [11]:
# Definimos el schems
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])

In [12]:
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)

In [45]:
## para guardarlo en memoria y que no se forme cadavez que se le llama e y las acciones vayayan mas rapido te imprime el schema del data frame
fire_df.cache()

DataFrame[CallNumber: int, UnitID: string, IncidentNumber: int, CallType: string, CallDate: string, WatchDate: string, CallFinalDisposition: string, AvailableDtTm: string, Address: string, City: string, Zipcode: int, Battalion: string, StationArea: string, Box: string, OriginalPriority: string, Priority: string, FinalPriority: int, ALSUnit: boolean, CallTypeGroup: string, NumAlarms: int, UnitType: string, UnitSequenceInCallDispatch: int, FirePreventionDistrict: string, SupervisorDistrict: string, Neighborhood: string, Location: string, RowID: string, Delay: float]

In [46]:
## Contar las lineas
fire_df.count()

163434

In [47]:
## para ver el schema del data frame de foma esquematica
fire_df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [48]:
## para ver el schema del data frame en una linea y pone nombre : tipo
## (nullable = true) es que permite que datos sean null
display(fire_df.limit(5))

DataFrame[CallNumber: int, UnitID: string, IncidentNumber: int, CallType: string, CallDate: string, WatchDate: string, CallFinalDisposition: string, AvailableDtTm: string, Address: string, City: string, Zipcode: int, Battalion: string, StationArea: string, Box: string, OriginalPriority: string, Priority: string, FinalPriority: int, ALSUnit: boolean, CallTypeGroup: string, NumAlarms: int, UnitType: string, UnitSequenceInCallDispatch: int, FirePreventionDistrict: string, SupervisorDistrict: string, Neighborhood: string, Location: string, RowID: string, Delay: float]

In [49]:
##  llamamos al df y luego ponemos .where(col("_______") ==/!=/</>/>=/<= "categoria" o valor))
## truncate hace que salga la fila entera o no si es larga:  fl-> |27th Av. / Cabrillo St.|   tr-> |27th Av. / Cabril...|
few_fire_df = (fire_df.select("IncidentNumber", "AvailableDtTm", "CallType") 
              .where(col("CallType") != "Medical Incident"))

few_fire_df.show(5, truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [50]:
## How many distinct CallTypes were recorded as the causes of the fire calls?
## .distinct() 
##
fire_df.select("CallType").where(col("CallType").isNotNull()).distinct().count()

30

In [51]:
## We can list the distinct call types in the data set using these queries:
fire_df.select("CallType").where(col("CallType").isNotNull()).distinct().show(10, False)

+-----------------------------+
|CallType                     |
+-----------------------------+
|Elevator / Escalator Rescue  |
|Marine Fire                  |
|Aircraft Emergency           |
|Administrative               |
|Alarms                       |
|Odor (Strange / Unknown)     |
|Citizen Assist / Service Call|
|HazMat                       |
|Watercraft in Distress       |
|Explosion                    |
+-----------------------------+
only showing top 10 rows



In [52]:
## Find out all response or delayed times greater than 5 mins?
new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")
new_fire_df.select( "ResponseDelayedinMins").where(col( "ResponseDelayedinMins") > 1).show(5, False)

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|2.95                 |
|4.7                  |
|2.4333334            |
|1.5                  |
|3.4833333            |
+---------------------+
only showing top 5 rows



In [53]:
new_fire_df.select(to_date(col("AvailableDtTm"), "MM/dd/yyyy").alias("IncidentDate")).show()

Py4JJavaError: An error occurred while calling o467.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 74) (L2110013.bosonit.local executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '01/11/2002 01:51:44 AM' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at 

In [36]:
df_fechas_tipo = df_firecalls.select(to_date(col("CallDate"), "yyyy-MM-ddThh:mm:ss").alias("date"), col("CallType"))
df_fechas_tipo.createOrReplaceTempView("tabla_fire_calls")

NameError: name 'df_firecalls' is not defined

In [69]:
## Covertimos las fechas en formato string a fecha
fire_ts_df = new_fire_df.select(to_date(col("CallDate"), "MM/dd/yyyy").alias("IncidentDate"),col("CallDate"),
                                to_date(col("WatchDate"), "MM/dd/yyyy").alias("OnWatchDate"),col("WatchDate"),
                                to_date(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a").alias("AvailableDtTS"),col("AvailableDtTm"))

fire_ts_df.select("IncidentDate","OnWatchDate","AvailableDtTS","CallDate","WatchDate","AvailableDtTm").show(3)

+------------+-----------+-------------+----------+----------+--------------------+
|IncidentDate|OnWatchDate|AvailableDtTS|  CallDate| WatchDate|       AvailableDtTm|
+------------+-----------+-------------+----------+----------+--------------------+
|  2002-01-11| 2002-01-10|   2002-01-11|01/11/2002|01/10/2002|01/11/2002 01:51:...|
|  2002-01-11| 2002-01-10|   2002-01-11|01/11/2002|01/10/2002|01/11/2002 03:01:...|
|  2002-01-11| 2002-01-10|   2002-01-11|01/11/2002|01/10/2002|01/11/2002 02:39:...|
+------------+-----------+-------------+----------+----------+--------------------+
only showing top 3 rows



In [56]:
#Guardamos en cache
fire_ts_df.cache()

fire_ts_df.columns

['IncidentDate',
 'CallDate',
 'OnWatchDate',
 'WatchDate',
 'AvailableDtTS',
 'AvailableDtTm']

In [76]:
#Check the transformed columns with Spark Timestamp type
fire_ts_df.select("IncidentDate", "OnWatchDate", "AvailableDtTS").where(col("IncidentDate").isNotNull()).show(30, False)

+------------+-----------+-------------+
|IncidentDate|OnWatchDate|AvailableDtTS|
+------------+-----------+-------------+
|2002-01-11  |2002-01-10 |2002-01-11   |
|2002-01-11  |2002-01-10 |2002-01-11   |
|2002-01-11  |2002-01-10 |2002-01-11   |
|2002-01-11  |2002-01-10 |2002-01-11   |
|2002-01-11  |2002-01-10 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |2002-01-11 |2002-01-11   |
|2002-01-11  |20

In [85]:
## What were the most common call types?
(new_fire_df
 .select("CallType").where(col("CallType").isNotNull())
 .groupBy("CallType")
 .count().withColumnRenamed("count",'numero de llamadas')
 .orderBy("numero de llamadas", ascending=False)
 .show(n=10, truncate=False))

+-------------------------------+------------------+
|CallType                       |numero de llamadas|
+-------------------------------+------------------+
|Medical Incident               |105513            |
|Structure Fire                 |22319             |
|Alarms                         |18094             |
|Traffic Collision              |6506              |
|Citizen Assist / Service Call  |2385              |
|Other                          |2024              |
|Outside Fire                   |1924              |
|Vehicle Fire                   |810               |
|Gas Leak (Natural and LP Gases)|700               |
|Water Rescue                   |696               |
+-------------------------------+------------------+
only showing top 10 rows



In [17]:
## What zip codes accounted for most common calls?
(fire_ts_df
 .select("CallType", "ZipCode")
 .where(col("CallType").isNotNull())
 .groupBy("CallType", "Zipcode")
 .count()
 .orderBy("count", ascending=False)
 .show(10, truncate=False))

+-----------------------------+-------+-----+
|CallType                     |Zipcode|count|
+-----------------------------+-------+-----+
|1 Sf Intl Airport            |94128  |6296 |
|1 SF INTL AIRPORT            |94128  |1710 |
|1001 POTRERO AVENUE          |94110  |457  |
|16th St. / Mission St.       |94110  |456  |
|22nd St. / Mission St.       |94110  |428  |
|21st St. / South Van Ness Av.|94110  |411  |
|865 MARKET STREET            |94103  |390  |
|7th St. / Mission St.        |94105  |355  |
|600 Jones St.                |94109  |327  |
|4th St. / Mission St.        |94105  |325  |
+-----------------------------+-------+-----+
only showing top 10 rows



In [87]:
## What San Francisco neighborhoods are in the zip codes 94102 and 94103
new_fire_df.select("Neighborhood", "Zipcode").where((col("Zipcode") == 94102) | (col("Zipcode") == 94103)).distinct().show(10, truncate=False)

+------------------------------+-------+
|Neighborhood                  |Zipcode|
+------------------------------+-------+
|Potrero Hill                  |94103  |
|Western Addition              |94102  |
|Tenderloin                    |94102  |
|Nob Hill                      |94102  |
|Castro/Upper Market           |94103  |
|South of Market               |94102  |
|South of Market               |94103  |
|Hayes Valley                  |94103  |
|Financial District/South Beach|94102  |
|Mission Bay                   |94103  |
+------------------------------+-------+
only showing top 10 rows



In [88]:
new_fire_df.select(sum("NumAlarms"), avg("ResponseDelayedinMins"), min("ResponseDelayedinMins"), max("ResponseDelayedinMins")).show()

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|        164292|         3.891114198630972|               0.016666668|                   1844.55|
+--------------+--------------------------+--------------------------+--------------------------+



In [20]:
fire_ts_df.select(year('IncidentDate')).distinct().orderBy(year('IncidentDate')).show()

+------------------+
|year(IncidentDate)|
+------------------+
|              null|
+------------------+



In [90]:
fire_ts_df.filter(year('IncidentDate') == 2018).groupBy(weekofyear('IncidentDate')).count().orderBy('count', ascending=False).show()

+------------------------+-----+
|weekofyear(IncidentDate)|count|
+------------------------+-----+
|                      13|  243|
|                      11|  240|
|                       5|  236|
|                       8|  232|
|                      10|  232|
|                       7|  228|
|                       9|  228|
|                      14|  225|
|                       6|  225|
|                      12|  221|
|                       4|  201|
|                      15|  193|
|                      44|  165|
|                       3|  100|
|                      36|   66|
|                      45|   64|
|                      38|   51|
|                      39|   50|
|                      40|   35|
|                      37|    8|
+------------------------+-----+
only showing top 20 rows



In [94]:
fire_ts_df.filter(year("IncidentDate") == 2001).show(10, False)

+------------+----------+-----------+----------+-------------+----------------------+
|IncidentDate|CallDate  |OnWatchDate|WatchDate |AvailableDtTS|AvailableDtTm         |
+------------+----------+-----------+----------+-------------+----------------------+
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 12:55:21 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 12:43:12 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 01:28:04 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 01:45:54 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 02:10:47 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 02:05:38 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 02:46:15 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-01-01   |01/01/2001 03:08:20 AM|
|2001-01-01  |01/01/2001|2000-12-31 |12/31/2000|2001-0

In [110]:
fire_ts_df.write.format("csv").mode("overwrite").save("C:/Users/javier.martin/YYY").coalesce(1)

Py4JJavaError: An error occurred while calling o928.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)


In [ ]:
%fs ls /tmp/fireServiceParquet/

In [ ]:
fire_ts_df.write.format("parquet").mode("overwrite").saveAsTable("FireServiceCalls")

In [ ]:
%sql
CACHE TABLE FireServiceCalls

In [ ]:
%sql
SELECT * FROM FireServiceCalls LIMIT 10

In [ ]:
file_parquet_df = spark.read.format("parquet").load("/tmp/fireServiceParquet/")

In [ ]:
display(file_parquet_df.limit(10))
